# Using the Wolfram Alpha LLM API as a Tool with Claude
In this recipe, we'll show you how to integrate the Wolfram Alpha LLM API as a tool for Claude to use. Claude will be able to send queries to the Wolfram Alpha API and receive computed responses, which it can then use to provide answers to user questions.

## Step 1: Set up the environment
First, let's install the required libraries and set up the Anthropic API client. We also will need to set our APP ID for using WolframAlpha. You can sign up and create a new App ID for this project for free [here](https://developer.wolframalpha.com/access).

In [10]:
from anthropic import Anthropic
import requests
import urllib.parse
import json

client = Anthropic()

# Replace 'YOUR_APP_ID' with your actual Wolfram Alpha AppID
WOLFRAM_APP_ID = 'YOUR_APP_ID'

## Step 2: Define the Wolfram Alpha LLM API tool
We'll define a tool that allows Claude to send queries to the Wolfram Alpha LLM API and receive the computed response.

In [ ]:
def wolfram_alpha_query(query):    
    # URL-encode the query
    encoded_query = urllib.parse.quote(query)
    
    # Make a request to the Wolfram Alpha LLM API
    url = f'https://www.wolframalpha.com/api/v1/llm-api?input={encoded_query}&appid={WOLFRAM_APP_ID}'
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.text
    else:
        return f"Error: {response.status_code}: {response.text}"

tools = [
    {
        "name": "wolfram_alpha",
        "description": "A tool that allows querying the Wolfram Alpha knowledge base. Useful for mathematical calculations, scientific data, and general knowledge questions.",
        "input_schema": {
            "type": "object",
            "properties": {
                "search_query": {
                    "type": "string",
                    "description": "The query to send to the Wolfram Alpha API."
                }
            },
            "required": ["query"]
        }
    }
]

In this code, we define a wolfram_alpha_query function that takes a query as input, URL-encodes it, and sends a request to the Wolfram Alpha LLM API using the provided AppID. The function returns the computed response from the API if the request is successful, or an error message if there's an issue.

We then define the wolfram_alpha tool with an input schema that expects a single query property of type string.


## Step 3: Interact with Claude
Now, let's see how Claude can interact with the Wolfram Alpha tool to answer user questions.

In [27]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "wolfram_alpha":
        return wolfram_alpha_query(tool_input["search_query"])

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
    prompt = f"""Here is a question: {user_message}. Please use the Wolfram Alpha tool to answer it. Do not reflect on the quality of the returned search results in your response."""

    message = client.beta.tools.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=[{"role": "user", "content": prompt}]
    )
    
    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")
    
    if message.stop_reason == "tool_use":
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input
        
        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))
        
        tool_result = process_tool_call(tool_name, tool_input)
        
        print(f"\nTool Result:")
        print(str(json.dumps(tool_result, indent=2)))
    
        
        response = client.beta.tools.messages.create(
            model=MODEL_NAME,
            max_tokens=2000,
            tools=tools,
            messages=[
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": str(tool_result)
                        }
                    ]
                }
            ]
        )
        
        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")
    else:
        response = message
    
    final_response = None
    for block in response.content:
        if hasattr(block, 'text'):
            final_response = block.text
            break
    
    print(f"\nFinal Response: {final_response}")
    
    return final_response

## Step 4: Try it out!
Let's try giving Claude a few example questions now that it has access to Wolfram Alpha.

In [28]:
# Example usage
print(chat_with_claude("What are the 5 largest countries in the world by population?"))
print(chat_with_claude("Calculate the square root of 1764."))
print(chat_with_claude("What is the distance between Earth and Mars?"))


User Message: What are the 5 largest countries in the world by population?

Initial Response:
Stop Reason: tool_use
Content: [ContentBlock(text='<thinking>\nThe query "What are the 5 largest countries in the world by population?" can be answered well using the wolfram_alpha tool, which has knowledge about countries and populations. The search_query parameter is the only required parameter, and the query text provided by the user can be used directly as the search_query value without any additional information needed. \n</thinking>', type='text'), ContentBlockToolUse(id='toolu_01VCQ5xAzMNdyXYsepbSAJLY', input={'search_query': 'What are the 5 largest countries in the world by population?'}, name='wolfram_alpha', type='tool_use')]

Tool Used: wolfram_alpha
Tool Input:
{
  "search_query": "What are the 5 largest countries in the world by population?"
}

Tool Result:
"Query:\n\"What are the 5 largest countries in the world by population?\"\n\nInput interpretation:\n5 largest countries | by